In [135]:
#imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
import lxml

# Obtain Information for Iterative Scraping 

In [137]:
#go to Whiskey Advocate

booze = 'https://www.whiskyadvocate.com/'

#browser.visit(booze)

In [138]:
#scrape the style dropdown menu into a list

response = requests.get(booze)

soup = bs(response.text)

styles = soup.find_all('option')


print(len(styles))

cleanSty = []
id_vals = []

for style in styles:
    #print(style['value'])
    cleanSty.append(style.text)
    id_vals.append(style['value'])

1301


In [139]:
#use a dataframe to number these for easy identification

stDict = {"dropdown_options": cleanSty, "option_value": id_vals}

st_df = pd.DataFrame(stDict)

pd.set_option('display.max_rows', 400)

#st_df.tail(400)

In [140]:
#cut the list down to just the relevant values
#I most likely need the id values because they are what actually plug in the URL for the search url

w_styles = st_df.iloc[1142:1203]['dropdown_options'].to_list()

w_ids = st_df.iloc[1142:1203]['option_value'].to_list()

w_ids

['68',
 '15',
 '31',
 '52',
 '27',
 '53',
 '42',
 '57',
 '55',
 '67',
 '17',
 '69',
 '3',
 '4',
 '7',
 '10',
 '63',
 '40',
 '49',
 '48',
 '47',
 '44',
 '41',
 '28',
 '60',
 '19',
 '8',
 '65',
 '37',
 '46',
 '71',
 '72',
 '70',
 '11',
 '50',
 '73',
 '45',
 '62',
 '26',
 '9',
 '56',
 '5',
 '58',
 '20',
 '64',
 '18',
 '21',
 '23',
 '61',
 '59',
 '1',
 '16',
 '29',
 '32',
 '39',
 '30',
 '2',
 '13',
 '33',
 '34',
 '66']

# Identify Location of Relevant Information in the Search Results Page

In [141]:
#I will iterate through this list when I'm ready to scrape my full dataset. 
#search_url works with browser command

search_url ='https://www.whiskyadvocate.com/ratings-reviews/?search=&submit=&brand_id=0&rating=0&price=0&category=0&styles_id=68&issue_id=0'

#browser.visit(search_url)

In [142]:
#Now I need to figure out how to scrape the crap off the page at the test url. the first style has 8 entries 
#I should get 8 entries in the lists. 

chow = requests.get(search_url)

chowder = bs(chow.text)

scores = chowder.find_all('span', itemprop = 'ratingValue')

print(f'{len(scores)} scores found')

for score in scores:
    print(score.text)

8 scores found
88
87
87
86
85
84
83
82


In [143]:
#get whiskey name now

names = chowder.find_all('h1', itemprop = 'name')

print(f'{len(scores)} names found')

for name in names:
    print(name.text)

8 names found
Spirit Hound Single Barrel Straight Malt (Barrel 66), 45%
Dogfish Head Straight Malt, 40%
Balcones Texas Bock Straight Malt Whiskey, 50%
Woodford Reserve Straight Malt Whiskey, 45.2%
Spirit Hound Colorado Honey Malt (Barrel No. 4), 45%
Parker’s Heritage Collection Kentucky Straight Malt Whiskey 8 year old, 54%
Woodford Reserve Master’s Collection Straight Malt, 45.2%
Woodford Reserve Master’s Collection Classic Malt, 45.2%


In [144]:
#get the whiskey price now

price_int = chowder.find_all('span', itemprop = 'price')

curr_unit = chowder.find_all('span', itemprop = 'priceCurrency')

#note, the relevent information is stored in the 'content' field in both these cases
for curr in curr_unit:
    print(curr['content'])

USD
USD
USD
USD
USD
USD
USD
USD


In [145]:
#get category designation

cat_des = chowder.find_all('span', itemprop = 'category')

for cat in cat_des:
    print(cat.text)

American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey


In [146]:
#get blurb

blurb = chowder.find_all('div', class_ = 'review-text', itemprop = 'description')

print(f'{len(blurb)} blurbs found')

print(blurb[0].text)

8 blurbs found

Floral with some hoppy aromas to start, as well as lemon curd, vanilla, banana Runts, ripe pears, and amaretto. On the palate, gentle oakiness underpins banana pudding, whipped cream, spiced plum, amaretto, ginger candy, peaches, and chai spices of cardamom, clove, and cinnamon. Well-structured and delicate—add water sparingly.


In [147]:
#get reviewer

reviewer = chowder.find_all('span', itemprop = 'author')

for re in reviewer:
    print(re.text)

Susannah Skiver Barton
Susannah Skiver Barton
David Fleming
Jeffery Lindenmuth
Susannah Skiver Barton
John Hansell
Lew Bryson
Lew Bryson


# Write the scrape_whisky Function

In [148]:
#the Scrape_whisky function is for iterating through all the differrent style options to compile a full dataset
#the function should loop through all the styles of whisky

import time

def scrape_whisky(idArray):
    
    #set up lists for data from the site
    whis_name = []
    w_scores = []
    price_val = []
    price_curr = []
    w_cat = []
    reviews = []
    authors = []
    counter = 0
    
    #this for loop will be to loop through all the style designations
    
    for ids in idArray:
        #set up url
        loop_url = f'https://www.whiskyadvocate.com/ratings-reviews/?search=&submit=&brand_id=0&rating=0&price=0&category=0&styles_id={ids}&issue_id=0'
        
        #make the soup object
        bisque = requests.get(loop_url)
        lobster_bisque = bs(bisque.text)
        
        #scrape the data and loop through the scrapes, appending the relevant portions for each 
        nms = lobster_bisque.find_all('h1', itemprop = 'name')
        
        for n in nms:
            whis_name.append(n.text)
            
        scrs = lobster_bisque.find_all('span', itemprop = 'ratingValue')
        
        for s in scrs:
            w_scores.append(s.text)
        
        prc = lobster_bisque.find_all('span', itemprop = 'price')
        
        for p in prc:
            price_val.append(p.text)
        
        crncy = lobster_bisque.find_all('span', itemprop = 'priceCurrency')
        
        for c in crncy:
            price_curr.append(c['content'])
        
        ctgry = lobster_bisque.find_all('span', itemprop = 'category')
        
        for ct in ctgry:
            w_cat.append(ct.text)
        
        rvw = lobster_bisque.find_all('div', class_ = 'review-text', itemprop = 'description')
        
        for r in rvw:
            reviews.append(r.text)
        
        auts = lobster_bisque.find_all('span', itemprop = 'author')
        
        for au in auts:
            authors.append(au.text)
        
        #feedback
        counter += 1
        print(f'finished scrape {counter}')
        print(f'list lengths: {len(nms)}, {len(scrs)}, {len(prc)}, {len(crncy)}, {len(ctgry)}, {len(rvw)}, {len(auts)}')
        print(f'style id added: {ids}')
        
        #sleep for a little while to aviod slamming their site with too many requests.
        time.sleep(bisque.elapsed.total_seconds())
    
    #loop ends here: more feedback
    print('finished scraping')
    print(f'''list lengths are: Names: {len(whis_name)}, Scores: {len(w_scores)}, Prices: {len(price_val)}, 
          Currency: {len(price_curr)}, Category:{len(w_cat)}, Reviews: {len(reviews)}, Author: {len(authors)}''' )
              
    #package data into a dictionary
    whisDict = {
        'name': whis_name, 'score': w_scores, 'price': price_val, 'currency_unit': price_curr, 'category': w_cat,
        'review': reviews, 'author' : authors
    }
    
    return whisDict


W_dict = scrape_whisky(w_ids)    


finished scrape 1
list lengths: 8, 8, 8, 8, 8, 8, 8
style id added: 68
finished scrape 2
list lengths: 58, 58, 58, 58, 58, 58, 58
style id added: 15
finished scrape 3
list lengths: 46, 46, 46, 46, 46, 46, 46
style id added: 31
finished scrape 4
list lengths: 11, 11, 11, 11, 11, 11, 11
style id added: 52
finished scrape 5
list lengths: 9, 9, 9, 9, 9, 9, 9
style id added: 27
finished scrape 6
list lengths: 1, 1, 1, 1, 1, 1, 1
style id added: 53
finished scrape 7
list lengths: 33, 33, 33, 33, 33, 33, 33
style id added: 42
finished scrape 8
list lengths: 1, 1, 1, 1, 1, 1, 1
style id added: 57
finished scrape 9
list lengths: 3, 3, 3, 3, 3, 3, 3
style id added: 55
finished scrape 10
list lengths: 9, 9, 9, 9, 9, 9, 9
style id added: 67
finished scrape 11
list lengths: 115, 115, 115, 115, 115, 115, 115
style id added: 17
finished scrape 12
list lengths: 2, 2, 2, 2, 2, 2, 2
style id added: 69
finished scrape 13
list lengths: 190, 190, 190, 190, 190, 190, 190
style id added: 3
finished scrape 14

In [149]:
#try encoding the reviews in UTF-8 to fix character display bug
#the important part of this is the utf-8-sig part the website was encoded in utf-8 and that's what bs4 and python were using
#but excel was apparently having a hard time with that detal.


dbg = W_dict['review'].copy()

dbg = [r.strip('\n') for r in dbg]


dbg_dict = {'text' : dbg}

dbg_df = pd.DataFrame(dbg_dict)


In [150]:
Whis_df = pd.DataFrame(W_dict)

Whis_df.head()

,name,score,price,Currency_unit,category,review,author
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,USD,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton
1,"Dogfish Head Straight Malt, 40%",87,40,USD,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,USD,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,USD,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,USD,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton


In [151]:
#write out the dataset as a csv for a convenient save point

#Whis_df.to_csv('desktop/independant_projects/whiskey_data_scraper/rough_whiskey_data.csv')

In [152]:
print(W_dict['review'][0])


Floral with some hoppy aromas to start, as well as lemon curd, vanilla, banana Runts, ripe pears, and amaretto. On the palate, gentle oakiness underpins banana pudding, whipped cream, spiced plum, amaretto, ginger candy, peaches, and chai spices of cardamom, clove, and cinnamon. Well-structured and delicate—add water sparingly.


In [153]:
#groupby currency to see if there is anything other than USD

currency_chck = Whis_df.groupby('currency_unit').count()

currency_chck

,name,score,price,category,review,author
Currency_unit,,,,,,
USD,5905,5905,5905,5905,5905,5905


The Currency_unit column can be dropped: its all USD

In [154]:
Whis_df = Whis_df.drop(columns = ['currency_unit'])

Whis_df.head()

,name,score,price,category,review,author
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton
1,"Dogfish Head Straight Malt, 40%",87,40,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton


In [155]:
Whis_df = Whis_df.rename(columns = {'price': 'price_usd'})

Whis_df.head()

,name,score,price_usd,category,review,author
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton
1,"Dogfish Head Straight Malt, 40%",87,40,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton


In [156]:
Whis_df.dtypes

name         object
score        object
price_usd    object
category     object
review       object
author       object
dtype: object

In [157]:
Whis_df['score'] = pd.to_numeric(Whis_df['score'])

Whis_df.dtypes

name         object
score         int64
price_usd    object
category     object
review       object
author       object
dtype: object

In [158]:
#Grouping by Price, to check for problem values since I couldn't retype price

prc_chk = Whis_df.groupby('price_usd').count()

prc_chk

,name,score,category,review,author
price_usd,,,,,
$130,1,1,1,1,1
"1,000",2,2,2,2,2
"1,008",1,1,1,1,1
"1,052",1,1,1,1,1
"1,058",1,1,1,1,1
...,...,...,...,...,...
98.00,3,3,3,3,3
989,2,2,2,2,2
99,39,39,39,39,39


Things to do with the price column:

- Remove dollar signs and commas.

- split off a new column using / as a delimiter - This is being used in cases when the amount of whisky isn't 750 ml.

- numericize the column.

When the new column is made I can replace NaN values with 750 and drop whiskeys sold as a set, or do something else with them. 
The name column also needs to be split to remove alcohol percentage to a new column. I will also have to put up with NaN values 
in that column. 

In [159]:
#get rid of $ sign and comma

Whis_df['price_usd'] = Whis_df['price_usd'].str.replace('$', '')
Whis_df['price_usd'] = Whis_df['price_usd'].str.replace(',', '')
prc_chk = Whis_df.groupby('price_usd').count()

prc_chk

,name,score,category,review,author
price_usd,,,,,
10,7,7,7,7,7
10.00,1,1,1,1,1
100,184,184,184,184,184
100.00,41,41,41,41,41
100/375ml.,1,1,1,1,1
...,...,...,...,...,...
98.00,3,3,3,3,3
989,2,2,2,2,2
99,39,39,39,39,39


In [160]:
#split the price column with / as the delimiter

pr_list = Whis_df['price_usd'].to_list()
clean_price = []
volume = []

for pr in pr_list:
    split_list = pr.split('/')
    clean_price.append(split_list[0])
    try:
        volume.append(split_list[1])
    except:
        volume.append(750)

Whis_df['price_usd'] = clean_price

Whis_df['sale_volume'] = volume

Whis_df.head()
    

,name,score,price_usd,category,review,author,sale_volume
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750
1,"Dogfish Head Straight Malt, 40%",87,40,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750


In [161]:
#I tried to set the value to numeric, it tripped up on line 3527
# I can't use this because the price is uncertain
Whis_df.iloc[3527]

name                        The Balvenie 1973 43 year old, 46.6%
score                                                         95
price_usd                                         15000 or 60000
category                                      Single Malt Scotch
review         \nThis expression was matured in a European oa...
author                                               Gavin Smith
sale_volume                                                  set
Name: 3527, dtype: object

In [162]:
#drop this row and confirm the drop
Whis_df = Whis_df.drop([3527])

Whis_df.iloc[3527]

name                           Glenglassaugh, 40 year old, 44.6%
score                                                         95
price_usd                                                2525.00
category                                      Single Malt Scotch
review         \nAn excellent example of an ultra-mature, she...
author                                              John Hansell
sale_volume                                                  750
Name: 3528, dtype: object

In [163]:
#retry the datatype reassignment

Whis_df['price_usd'] = pd.to_numeric(Whis_df['price_usd'])

Whis_df.dtypes

name            object
score            int64
price_usd      float64
category        object
review          object
author          object
sale_volume     object
dtype: object

In [164]:
# Split out ABV
# In cases where ABV is unknown, I will assume it is 40% which is the modal value for hard liquor

name_list = Whis_df['name'].to_list()

whis_names = []
abv = []

for name in name_list:
    nl = name.split(',')
    whis_names.append(nl[0])
    try:
        abv.append(nl[-1].strip())
    except:
        abv.append(40)

#Whis_df['Name'] = whis_names

Whis_df['percent_abv'] = abv

Whis_df.head()

,name,score,price_usd,category,review,author,sale_volume,percent_abv
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45%
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40%
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50%
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2%
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45%


In [165]:
#Get rid of the percent signs in the ABV column

Whis_df['percent_abv'] = Whis_df['percent_abv'].str.replace('%', '')

Whis_df.head()

,name,score,price_usd,category,review,author,sale_volume,percent_abv
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45


Basic procedure from here and below: I find an error where the last comma doesn't cleanly lead to a number representing ABV, and I make a change to the value. I run the iloc value first and then insert the change above it to save space. A lot of the time these are quotation marks getting pulled into the ABV value becuase of illogical English punctuation rules.

# Cleaning the ABV column

Line by line cleaning of broken ABV values

In [166]:
#Set Percent_ABV to a numeric datatype - didn't work
#check out position 55

Whis_df.iloc[55]

name           Finger Lakes Distilling McKenzie Pure Pot Stil...
score                                                         83
price_usd                                                     43
category                                        American Whiskey
review         \n\nMade from unmalted barley, malted barley, ...
author                                               Ted Simmons
sale_volume                                                  750
percent_abv    Finger Lakes Distilling McKenzie Pure Pot Stil...
Name: 55, dtype: object

In [167]:
#Rename by concatenating, and set ABV to default value of 40%

Whis_df.iloc[55, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[55]

name           Finger Lakes Distilling McKenzie Pure Pot Stil...
score                                                         83
price_usd                                                     43
category                                        American Whiskey
review         \n\nMade from unmalted barley, malted barley, ...
author                                               Ted Simmons
sale_volume                                                  750
percent_abv                                                   40
Name: 55, dtype: object

In [168]:
#Set Percent_ABV to a numeric datatype - didn't work
#check out position 133

Whis_df.iloc[133, Whis_df.columns.get_loc('percent_abv')] = 60.2

Whis_df.iloc[133]

name                     Little Book Chapter 1 “The Easy,” 60.2%
score                                                         92
price_usd                                                     80
category                                Blended American Whiskey
review         \nBy focusing exclusively on high-quality stra...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                 60.2
Name: 133, dtype: object

In [169]:
Whis_df.iloc[135, Whis_df.columns.get_loc('percent_abv')] = 60.2

Whis_df.iloc[135]

name                     Little Book Chapter 1 "The Easy," 60.2%
score                                                         91
price_usd                                                     80
category                                Blended American Whiskey
review         \nIt opens with dark chocolate, coffee, hazeln...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                 60.2
Name: 135, dtype: object

In [170]:
Whis_df.iloc[174, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[174]

name                                Compass Box Hedonism Maximus
score                                                         90
price_usd                                                    300
category                             Blended Grain Scotch Whisky
review         \nAn older, more exotic expression of Hedonism...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 174, dtype: object

In [171]:
Whis_df.iloc[790, Whis_df.columns.get_loc('percent_abv')] = 65.4
Whis_df.iloc[790]

name            Booker’s 2014-01 “25th Anniversary Batch,” 65.4%
score                                                         96
price_usd                                                    100
category                                       Bourbon/Tennessee
review         \nThe complete package: uncut, unfiltered, ful...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 65.4
Name: 790, dtype: object

In [172]:
Whis_df.iloc[795, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[795]

name                                            Pappy Van Winkle
score                                                         96
price_usd                                                    220
category                                       Bourbon/Tennessee
review         \nMy review of this whiskey a few years back i...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 795, dtype: object

In [173]:
Whis_df.iloc[805, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[805]

name                                         Old Rip Van Winkle 
score                                                         95
price_usd                                                    350
category                                       Bourbon/Tennessee
review         \nDistilled in 1986. An excellent old wheated ...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 805, dtype: object

In [174]:
Whis_df.iloc[814, Whis_df.columns.get_loc('percent_abv')] = 45

Whis_df.iloc[814]

name           Buffalo Trace Experimental Collection, 1995 Vi...
score                                                         95
price_usd                                                     47
category                                       Bourbon/Tennessee
review         \nSurprisingly light and fresh for a 15 year o...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   45
Name: 814, dtype: object

In [175]:
Whis_df.iloc[856, Whis_df.columns.get_loc('percent_abv')] = 60.20

Whis_df.iloc[856]

name           Bardstown Bourbon Co. Copper & Kings Apple Bra...
score                                                         94
price_usd                                                    125
category                                       Bourbon/Tennessee
review         \nAromas of corn meal, corn muffins, and butte...
author                                             David Fleming
sale_volume                                                  750
percent_abv                                                 60.2
Name: 856, dtype: object

In [176]:
Whis_df.iloc[952, Whis_df.columns.get_loc('percent_abv')] = 64.4

Whis_df.iloc[952]

name                      Booker’s 2018-02 “Backyard BBQ,” 64.4%
score                                                         93
price_usd                                                     75
category                                       Bourbon/Tennessee
review         \nTypical of Booker’s this bourbon shows lots ...
author                                        Jeffery Lindenmuth
sale_volume                                                  750
percent_abv                                                 64.4
Name: 952, dtype: object

In [177]:
Whis_df.iloc[969, Whis_df.columns.get_loc('percent_abv')] = 63.35

Whis_df.iloc[969]

name                    Booker’s 2018-03 “Kentucky Chew,” 63.35%
score                                                         93
price_usd                                                     70
category                                       Bourbon/Tennessee
review         \nOriginally touted by Booker Noe, the “Kentuc...
author                                    Susannah Skiver Barton
sale_volume                                                  750
percent_abv                                                63.35
Name: 969, dtype: object

In [178]:
Whis_df.iloc[978, Whis_df.columns.get_loc('percent_abv')] = 61.3

Whis_df.iloc[978]

name                Little Book Chapter 3 “The Road Home,” 61.3%
score                                                         93
price_usd                                                    125
category                                       Bourbon/Tennessee
review         \nThe chewy caramel aroma is bursting with all...
author                                        Jeffery Lindenmuth
sale_volume                                                  750
percent_abv                                                 61.3
Name: 978, dtype: object

In [179]:
Whis_df.iloc[991, Whis_df.columns.get_loc('percent_abv')] = 63.7

Whis_df.iloc[991]

name                  Booker’s 2018-01 "Kathleen’s Batch," 63.7%
score                                                         92
price_usd                                                     70
category                                       Bourbon/Tennessee
review         \nBright and pronounced caramel in many forms,...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                 63.7
Name: 991, dtype: object

In [180]:
Whis_df.iloc[1022, Whis_df.columns.get_loc('percent_abv')] = 64.25

Whis_df.iloc[1022]

name                    Booker’s 2017-01 “Tommy’s Batch,” 64.25%
score                                                         92
price_usd                                                     70
category                                       Bourbon/Tennessee
review         \nAt 6 years, 4 months, and 6 days, this is be...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                64.25
Name: 1022, dtype: object

In [181]:
Whis_df.iloc[1038, Whis_df.columns.get_loc('percent_abv')] = 64.05

Whis_df.iloc[1038]

name                       Booker’s 2017-04 “Sip Awhile,” 64.05%
score                                                         92
price_usd                                                     73
category                                       Bourbon/Tennessee
review         \nRosewater, almond extract, dark cherries, ro...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                64.05
Name: 1038, dtype: object

In [182]:
Whis_df.iloc[1041, Whis_df.columns.get_loc('percent_abv')] = 62.95

Whis_df.iloc[1041]

name                   Booker’s 2019-01 “Teresa’s Batch,” 62.95%
score                                                         92
price_usd                                                     80
category                                       Bourbon/Tennessee
review         \nOn the nose, there’s Golden Grahams, iced te...
author                                    Susannah Skiver Barton
sale_volume                                                  750
percent_abv                                                62.95
Name: 1041, dtype: object

In [183]:
Whis_df.iloc[1052, Whis_df.columns.get_loc('percent_abv')] = 64

Whis_df.iloc[1052]

name                       Booker’s 2018-04 “Kitchen Table,” 64%
score                                                         92
price_usd                                                     75
category                                       Bourbon/Tennessee
review         \nA classic bourbon nose, as butterscotch hard...
author                                        Jeffery Lindenmuth
sale_volume                                                  750
percent_abv                                                   64
Name: 1052, dtype: object

In [184]:
Whis_df.iloc[1093, Whis_df.columns.get_loc('percent_abv')] = 63.95

Whis_df.iloc[1093]

name                  Booker’s 2016-01 “Bluegrass Batch,” 63.95%
score                                                         91
price_usd                                                     60
category                                       Bourbon/Tennessee
review         \nIf Picasso sketched bourbon, it would look l...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                63.95
Name: 1093, dtype: object

In [185]:
Whis_df.iloc[1225, Whis_df.columns.get_loc('percent_abv')] = 43.3

Whis_df.iloc[1225]

name           Evan Williams Single Barrel 1994 Vintage (Barr...
score                                                         89
price_usd                                                     25
category                                       Bourbon/Tennessee
review         \nThis is an elegant whiskey with lots of fine...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 43.3
Name: 1225, dtype: object

In [186]:
Whis_df.iloc[1290, Whis_df.columns.get_loc('percent_abv')] = 46.2

Whis_df.iloc[1290]

name           Woodford Reserve Master’s Collection “New Cask...
score                                                         88
price_usd                                                    100
category                                       Bourbon/Tennessee
review         \nA set of two 100% rye whiskeys, triple disti...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 46.2
Name: 1290, dtype: object

In [187]:
Whis_df.iloc[1294, Whis_df.columns.get_loc('percent_abv')] = 50.2

Whis_df.iloc[1294]

name           Woodford Reserve Master's Collection 'Seasoned...
score                                                         88
price_usd                                                     90
category                                       Bourbon/Tennessee
review         \nThe fourth in the series of limited-edition ...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 50.2
Name: 1294, dtype: object

In [188]:
Whis_df.iloc[1294, Whis_df.columns.get_loc('percent_abv')] = 45

Whis_df.iloc[1294]

name           Woodford Reserve Master's Collection 'Seasoned...
score                                                         88
price_usd                                                     90
category                                       Bourbon/Tennessee
review         \nThe fourth in the series of limited-edition ...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   45
Name: 1294, dtype: object

In [189]:
Whis_df.iloc[1327, Whis_df.columns.get_loc('percent_abv')] = 45

Whis_df.iloc[1327]

name           Buffalo Trace Experimental Collection, 1995 Vi...
score                                                         88
price_usd                                                     47
category                                       Bourbon/Tennessee
review         \nDark, big, and brooding. A mouthful of oak s...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   45
Name: 1327, dtype: object

In [190]:
Whis_df.iloc[1332, Whis_df.columns.get_loc('percent_abv')] = 64.35

Whis_df.iloc[1332]

name             Booker’s 2015-01 “Big Man, Small Batch,” 64.35%
score                                                         88
price_usd                                                     55
category                                       Bourbon/Tennessee
review         \nThe first in a series of limited edition rel...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                64.35
Name: 1332, dtype: object

In [191]:
Whis_df.iloc[1364, Whis_df.columns.get_loc('percent_abv')] = 58

Whis_df.iloc[1364]

name           Four Roses Mariage Collection (2009 Release), ...
score                                                         87
price_usd                                                     80
category                                       Bourbon/Tennessee
review         \nA combination of 10 and 19 year old bourbons...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   58
Name: 1364, dtype: object

In [192]:
Whis_df.iloc[1368, Whis_df.columns.get_loc('percent_abv')] = 46.2

Whis_df.iloc[1368]

name           Woodford Reserve Master’s Collection “Aged Cas...
score                                                         87
price_usd                                                    100
category                                       Bourbon/Tennessee
review         \nA set of two 100% rye whiskeys, triple disti...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 46.2
Name: 1368, dtype: object

In [193]:
Whis_df.iloc[1403, Whis_df.columns.get_loc('percent_abv')] = 63.5

Whis_df.iloc[1403]

name                 Booker’s 2015-04 “Oven Buster Batch,” 63.5%
score                                                         87
price_usd                                                     60
category                                       Bourbon/Tennessee
review         \nAged over 6 years, this is a rather feminine...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 63.5
Name: 1403, dtype: object

In [194]:
Whis_df.iloc[1438, Whis_df.columns.get_loc('percent_abv')] = 63.7

Whis_df.iloc[1438]

name                Booker’s 2017-02 “Blue Knights Batch,” 63.7%
score                                                         86
price_usd                                                     75
category                                       Bourbon/Tennessee
review         \nThink corn: roasted, canned, creamed, mashed...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                 63.7
Name: 1438, dtype: object

In [195]:
Whis_df.iloc[1561, Whis_df.columns.get_loc('percent_abv')] = 45

Whis_df.iloc[1561]

name                                 Jim Beam “Devil’s Cut,” 45%
score                                                         83
price_usd                                                     24
category                                       Bourbon/Tennessee
review         \nMore wood influence than the standard Jim Be...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   45
Name: 1561, dtype: object

In [196]:
Whis_df.iloc[1590, Whis_df.columns.get_loc('percent_abv')] = 64.5

Whis_df.iloc[1590]

name               Booker’s 2016-03 “Toogie’s Invitation,” 64.5%
score                                                         82
price_usd                                                     60
category                                       Bourbon/Tennessee
review         \nAt 6 years, 4 months, and 4 days old, there’...
author                                              Fred Minnick
sale_volume                                                  750
percent_abv                                                 64.5
Name: 1590, dtype: object

In [197]:
Whis_df.iloc[1783, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[1783]

name                          Black Velvet Onyx 12 year old. 40%
score                                                         88
price_usd                                                     28
category                                                Canadian
review         \nDry and mildly dusty on first nosing, with a...
author                                      Davin de Kergommeaux
sale_volume                                                  750
percent_abv                                                   40
Name: 1783, dtype: object

In [198]:
Whis_df.iloc[1828, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[1828]

name                           Canadian Club Classic 12, 40% ABV
score                                                         85
price_usd                                                     22
category                                                Canadian
review         \nNose is much more integrated than the others...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                   40
Name: 1828, dtype: object

In [199]:
Whis_df.iloc[1841, Whis_df.columns.get_loc('percent_abv')] = 62.2

Whis_df.iloc[1841]

name                                  Glen Breton Ice, 62.2% ABV
score                                                         84
price_usd                                                     50
category                                                Canadian
review         \nAged 10 years in icewine barrels — a Canadia...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                 62.2
Name: 1841, dtype: object

In [200]:
Whis_df.iloc[1872, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[1872]

name                              Canadian Club Reserve, 40% ABV
score                                                         82
price_usd                                                     17
category                                                Canadian
review         \nA bit of heat, faint nose of putty, light br...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                   40
Name: 1872, dtype: object

In [201]:
Whis_df.iloc[1873, Whis_df.columns.get_loc('percent_abv')] = 41.3

Whis_df.iloc[1873]

name                        Canadian Club Sherry Cask, 41.3% ABV
score                                                         82
price_usd                                                     28
category                                                Canadian
review         \n8 years old and sherry finished, and it show...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                 41.3
Name: 1873, dtype: object

In [202]:
Whis_df.iloc[1892, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[1892]

name                                          Danfield's Private
score                                                         81
price_usd                                                     22
category                                                Canadian
review         \nLight in body and on the nose, but clean and...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 1892, dtype: object

In [203]:
Whis_df.iloc[1899, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[1899]

name                               Canadian Club Whisky, 40% ABV
score                                                         80
price_usd                                                     14
category                                                Canadian
review         \nPale, almost peach color. Light aromas of ca...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                   40
Name: 1899, dtype: object

In [204]:
Whis_df.iloc[1921, Whis_df.columns.get_loc('percent_abv')] = 50

Whis_df.iloc[1921]

name            Canadian Club 100 Proof Export Strength, 50% ABV
score                                                         77
price_usd                                                     20
category                                                Canadian
review         \nHot solvents on the nose, reminiscent of Hin...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                   50
Name: 1921, dtype: object

In [205]:
Whis_df.iloc[2161, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[2161]

name                 Wyoming Double Cask Sherry Finished Bourbon
score                                                         83
price_usd                                                     60
category                                           Craft Whiskey
review         \nPale gold in color. Generous oak imparts hon...
author                                        Jeffery Lindenmuth
sale_volume                                                  750
percent_abv                                                   40
Name: 2161, dtype: object

In [206]:
Whis_df.iloc[2364, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[2364]

name                                                    JP Wiser
score                                                         81
price_usd                                                     21
category                              Flavored Whisky & Liqueurs
review         \nBurnt caramel, sweet—almost candied—but not ...
author                                      Davin de Kergommeaux
sale_volume                                                  750
percent_abv                                                   40
Name: 2364, dtype: object

In [207]:
Whis_df.iloc[2409, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[2409]

name                                           Kornog Toal Esa Z
score                                                         90
price_usd                                                     77
category                                           French Whisky
review         \nKornog is the peated version of Glann ar Mor...
author                                           Dominic Roskrow
sale_volume                                                  750
percent_abv                                                   40
Name: 2409, dtype: object

In [208]:
Whis_df.iloc[2420, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[2420]

name                                              Kornog Taouarc
score                                                         88
price_usd                                                     77
category                                           French Whisky
review         \nOf all the releases I've tasted from this di...
author                                           Dominic Roskrow
sale_volume                                                  750
percent_abv                                                   40
Name: 2420, dtype: object

In [209]:
Whis_df.iloc[2479, Whis_df.columns.get_loc('percent_abv')] = 50

Whis_df.iloc[2479]

name                                         Amrut 'Fusion,' 50%
score                                                         89
price_usd                                                     50
category                                           Indian Whisky
review         \nBalance, complexity, and surprising maturity...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   50
Name: 2479, dtype: object

In [210]:
Whis_df.iloc[2778, Whis_df.columns.get_loc('percent_abv')] = 50

Whis_df.iloc[2778]

name                                        Fuji-Sanroku 50° 50%
score                                                         92
price_usd                                                     53
category                                         Japanese Whisky
review         \nJota Tanaka, Kirin’s chief blender, utilizes...
author                                           Jonny McCormick
sale_volume                                                  750
percent_abv                                                   50
Name: 2778, dtype: object

In [211]:
Whis_df.iloc[2791, Whis_df.columns.get_loc('percent_abv')] = 43

Whis_df.iloc[2791]

name                           Mars The Lucky Cat “Ash ’99,” 43%
score                                                         91
price_usd                                                    100
category                                         Japanese Whisky
review         \nYes, there is a cute, mewing black kitten on...
author                                           Jonny McCormick
sale_volume                                                  750
percent_abv                                                   43
Name: 2791, dtype: object

In [212]:
Whis_df.iloc[2794, Whis_df.columns.get_loc('percent_abv')] = 58

Whis_df.iloc[2794]

name           Ichiro’s Malt, Hanyu 23 year old “sherry casks...
score                                                         91
price_usd                                                    695
category                                         Japanese Whisky
review         \nThis oldie (from a distillery which closed i...
author                                                Dave Broom
sale_volume                                                  750
percent_abv                                                   58
Name: 2794, dtype: object

In [213]:
Whis_df.iloc[2828, Whis_df.columns.get_loc('percent_abv')] = 60.7

Whis_df.iloc[2828]

name           Karuizawa cask 4592, 1977 (bottle 2010), ‘Noh ...
score                                                         89
price_usd                                                    200
category                                         Japanese Whisky
review         \nThe (sadly mothballed) Karuizawa distillery ...
author                                                Dave Broom
sale_volume                                                  750
percent_abv                                                 60.7
Name: 2828, dtype: object

In [214]:
Whis_df.iloc[2902, Whis_df.columns.get_loc('percent_abv')] = 59.4

Whis_df.iloc[2902]

name              Little Book Chapter 2 “Noe Simple Task,” 59.4%
score                                                         93
price_usd                                                    100
category                                           Miscellaneous
review         \nA fascinating flurry of aromas—red fruit pun...
author                                        Jeffery Lindenmuth
sale_volume                                                  750
percent_abv                                                 59.4
Name: 2902, dtype: object

In [215]:
#row 2903 is a dupe

Whis_df = Whis_df.drop([2903])

Whis_df.iloc[2903]

name                         Barrell Whiskey (Batch 001), 61.25%
score                                                         92
price_usd                                                     59
category                                           Miscellaneous
review         \n7 year old whiskey (an unspecified “corn, ry...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                61.25
Name: 2904, dtype: object

In [216]:
Whis_df.iloc[2952, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[2952]

name                                                 New Zealand
score                                                         88
price_usd                                                    233
category                                             New Zealand
review         \nRugby union fans will recognize the name and...
author                                           Dominic Roskrow
sale_volume                                                  750
percent_abv                                                   40
Name: 2953, dtype: object

In [217]:
Whis_df.iloc[2960, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[2960]

name                                         Thomson 17 year old
score                                                         79
price_usd                                                     95
category                                             New Zealand
review         \nThomson has discovered and bottled stocks fr...
author                                           Dominic Roskrow
sale_volume                                                  750
percent_abv                                                   40
Name: 2961, dtype: object

In [218]:
Whis_df.iloc[3131, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[3131]

name                         Minor Case Sherry Cask Finished Rye
score                                                         87
price_usd                                                     50
category                                             Rye Whiskey
review         \nThe sherry cask finish asserts itself right ...
author                                    Susannah Skiver Barton
sale_volume                                                  750
percent_abv                                                   40
Name: 3132, dtype: object

In [219]:
Whis_df.iloc[3225, Whis_df.columns.get_loc('percent_abv')] = 46.5

Whis_df.iloc[3225]

name                          Grand Traverse "Ole George," 46.5%
score                                                         80
price_usd                                                     48
category                                             Rye Whiskey
review         \nSweet (caramel, butterscotch), botanical, an...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 46.5
Name: 3226, dtype: object

In [220]:
Whis_df.iloc[3437, Whis_df.columns.get_loc('percent_abv')] = 46.5

Whis_df.iloc[3437]

name           Woodstone Creek Single Malt “The Murray Barrel...
score                                                         83
price_usd                                                    118
category                            Single Malt American Whiskey
review         \nNice color for a 10 year old. Puffing peat o...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                 46.5
Name: 3438, dtype: object

In [221]:
Whis_df.iloc[3442, Whis_df.columns.get_loc('percent_abv')] = 44.4

Whis_df.iloc[3442]

name                                        The Notch, 44.4% ABV
score                                                         82
price_usd                                                    888
category                            Single Malt American Whiskey
review         \nThe price, the proof, and 8 years of age ref...
author                                                Lew Bryson
sale_volume                                                  750
percent_abv                                                 44.4
Name: 3443, dtype: object

In [222]:
Whis_df.iloc[3485, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[3485]

name                          St. George’s Chapter 6, 3 year old
score                                                         80
price_usd                                                     59
category                              Single Malt English Whisky
review         \nEngland’s sole whisky distillery (currently,...
author                                                Dave Broom
sale_volume                                                  750
percent_abv                                                   40
Name: 3486, dtype: object

In [223]:
Whis_df.iloc[3669, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[3669]

name           The Balvenie 30 year old 1973 Vintage (Cask #9...
score                                                         93
price_usd                                                    400
category                                      Single Malt Scotch
review         \nThe last Balvenie Vintage whisky I tasted th...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 3671, dtype: object

In [224]:
Whis_df.iloc[3677, Whis_df.columns.get_loc('percent_abv')] = 46.7

Whis_df.iloc[3677]

name                         The Glenrothes “John Ramsay,” 46.7%
score                                                         93
price_usd                                                   1000
category                                      Single Malt Scotch
review         \nMade from whisky aged in second fill America...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 46.7
Name: 3679, dtype: object

In [225]:
Whis_df.iloc[3731, Whis_df.columns.get_loc('percent_abv')] = 46

Whis_df.iloc[3731]

name                          Kilchoman 'Inaugural Release,' 46%
score                                                         92
price_usd                                                     60
category                                      Single Malt Scotch
review         \nAged 2.5 to 3 years on bourbon casks and the...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   46
Name: 3733, dtype: object

In [226]:
Whis_df.iloc[3731, Whis_df.columns.get_loc('percent_abv')] = 46

Whis_df.iloc[3731]

name                          Kilchoman 'Inaugural Release,' 46%
score                                                         92
price_usd                                                     60
category                                      Single Malt Scotch
review         \nAged 2.5 to 3 years on bourbon casks and the...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   46
Name: 3733, dtype: object

In [227]:
Whis_df.iloc[3774, Whis_df.columns.get_loc('percent_abv')] = 55

Whis_df.iloc[3774]

name                           Highland Park “Saint Magnus,” 55%
score                                                         92
price_usd                                                    132
category                                      Single Malt Scotch
review         \nThe second in a series of three high-strengt...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   55
Name: 3776, dtype: object

In [228]:
Whis_df.iloc[4263, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[4263]

name                                  Glenfarclas £511. 19s. 0d.
score                                                         89
price_usd                                                    130
category                                      Single Malt Scotch
review         \nUnusually named—it refers to the price paid ...
author                                                Dave Broom
sale_volume                                                  750
percent_abv                                                   40
Name: 4265, dtype: object

In [229]:
Whis_df.iloc[4277, Whis_df.columns.get_loc('percent_abv')] = 46

Whis_df.iloc[4277]

name                         Old Pulteney WK209 ‘Good Hope,’ 46%
score                                                         89
price_usd                                                     63
category                                      Single Malt Scotch
review         \nWK209 is named after a steam herring drifter...
author                                               Gavin Smith
sale_volume                                                  750
percent_abv                                                   46
Name: 4279, dtype: object

In [230]:
Whis_df.iloc[4316, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[4316]

name                                             Murray McDavid 
score                                                         89
price_usd                                                    150
category                                      Single Malt Scotch
review         \nYellow gold color. Fresh, floral aromas (rem...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 4318, dtype: object

In [231]:
Whis_df.iloc[4366, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[4366]

name                                                   Cadenhead
score                                                         89
price_usd                                                    125
category                                      Single Malt Scotch
review         \nAntique gold color. Aromas of peat smoke, da...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 4368, dtype: object

In [232]:
Whis_df.iloc[4389, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[4389]

name           Wemyss Malts ‘Honey Spice’ (distilled at Bunna...
score                                                         88
price_usd                                                     96
category                                      Single Malt Scotch
review         \nBunnahabhain, along with Bowmore, is Islay’s...
author                                                Dave Broom
sale_volume                                                  750
percent_abv                                                   40
Name: 4391, dtype: object

In [233]:
Whis_df.iloc[4410, Whis_df.columns.get_loc('percent_abv')] = 52.3

Whis_df.iloc[4410]

name                       Arran ‘The Devil’s Punch Bowl,’ 52.3%
score                                                         88
price_usd                                                    130
category                                      Single Malt Scotch
review         \n6,660 bottles of The Devil’s Punch Bowl have...
author                                               Gavin Smith
sale_volume                                                  750
percent_abv                                                 52.3
Name: 4412, dtype: object

In [234]:
Whis_df.iloc[4551, Whis_df.columns.get_loc('percent_abv')] = 45

Whis_df.iloc[4551]

name           Blackadder Single Cask #30013 (distilled at Li...
score                                                         88
price_usd                                                    135
category                                      Single Malt Scotch
review         \nAlso known as St. Magdalene, this Lowland di...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   45
Name: 4553, dtype: object

In [235]:
Whis_df.iloc[4654, Whis_df.columns.get_loc('percent_abv')] = 43

Whis_df.iloc[4654]

name                     Royal Lochnagar 'Selected Reserve,' 43%
score                                                         87
price_usd                                                    210
category                                      Single Malt Scotch
review         \nSo nice to see this whisky available here in...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   43
Name: 4656, dtype: object

In [236]:
Whis_df.iloc[4662, Whis_df.columns.get_loc('percent_abv')] = 43

Whis_df.iloc[4662]

name                              Glenrothes, 1994 Vintage, $43%
score                                                         87
price_usd                                                     78
category                                      Single Malt Scotch
review         \nVery bright and lively, with a nice balance ...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   43
Name: 4664, dtype: object

In [237]:
Whis_df.iloc[4667, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[4667]

name                               Ardmore Traditional Cask 1998
score                                                         87
price_usd                                                     50
category                                      Single Malt Scotch
review         \nA single cask of Ardmore aged in a first-fil...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 4669, dtype: object

In [238]:
Whis_df.iloc[4756, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[4756]

name           Cask & Thistle 29 year old 1974 Vintage (disti...
score                                                         87
price_usd                                                    110
category                                      Single Malt Scotch
review         \nA prime component in Dewar’s blended scotch....
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 4758, dtype: object

In [239]:
Whis_df.iloc[4771, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[4771]

name           Wild Scotsman (distilled at Ben Nevis), 13 yea...
score                                                         87
price_usd                                                     70
category                                      Single Malt Scotch
review         \nVery robust and chewy-this whisky has plenty...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 4773, dtype: object

In [240]:
Whis_df.iloc[4987, Whis_df.columns.get_loc('percent_abv')] = 45

Whis_df.iloc[4987]

name           Blackadder Single Cask #1101 (distilled at Lon...
score                                                         86
price_usd                                                    135
category                                      Single Malt Scotch
review         \nA big and sappy Longmorn, with notes of chew...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   45
Name: 4989, dtype: object

In [241]:
Whis_df.iloc[5023, Whis_df.columns.get_loc('percent_abv')] = 46

Whis_df.iloc[5023]

name           Wemyss Malts (distilled at Glen Scotia) 'Straw...
score                                                         85
price_usd                                                    174
category                                      Single Malt Scotch
review         \nThis single cask variant of Glen Scotia has ...
author                                               Gavin Smith
sale_volume                                                  750
percent_abv                                                   46
Name: 5025, dtype: object

In [242]:
Whis_df.iloc[5040, Whis_df.columns.get_loc('percent_abv')] = 43

Whis_df.iloc[5040]

name           Dun Bheagan (distilled at Caol Ila) 1993 Vinta...
score                                                         85
price_usd                                                     56
category                                      Single Malt Scotch
review         \nAn unusual Caol Ila, in that it was finished...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   43
Name: 5042, dtype: object

In [243]:
Whis_df.iloc[5116, Whis_df.columns.get_loc('percent_abv')] = 43

Whis_df.iloc[5116]

name                       The Glenrothes, 1992 Vintage, 43% ABV
score                                                         85
price_usd                                                     55
category                                      Single Malt Scotch
review         \nA very fragrant, fruity whisky (orange, tang...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   43
Name: 5118, dtype: object

In [244]:
Whis_df.iloc[5138, Whis_df.columns.get_loc('percent_abv')] = 56.4

Whis_df.iloc[5138]

name           Gordon & MacPhail Single Cask #12274 (distille...
score                                                         85
price_usd                                                     65
category                                      Single Malt Scotch
review         \nArdmore whiskies I’ve enjoyed in the past ha...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 56.4
Name: 5140, dtype: object

In [245]:
Whis_df.iloc[5140, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[5140]

name           Duncan Taylor NC2 (distilled at Imperial) 1997...
score                                                         85
price_usd                                                     60
category                                      Single Malt Scotch
review         \nPale straw. One of a seemingly never-ending ...
author                                                Dave Broom
sale_volume                                                  750
percent_abv                                                   40
Name: 5142, dtype: object

In [246]:
Whis_df.iloc[5154, Whis_df.columns.get_loc('percent_abv')] = 59.1

Whis_df.iloc[5154]

name           Montgomerie's Single Cask #910 (distilled at B...
score                                                         85
price_usd                                                     65
category                                      Single Malt Scotch
review         \n(Exclusive to Park Ave Liquors, New York, NY...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 59.1
Name: 5156, dtype: object

In [247]:
Whis_df.iloc[5198, Whis_df.columns.get_loc('percent_abv')] = 57.9

Whis_df.iloc[5198]

name           Gordon & MacPhail Single Cask #1961 (distilled...
score                                                         84
price_usd                                                     65
category                                      Single Malt Scotch
review         \nIt is unusual to find a Bruichladdich whisky...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 57.9
Name: 5200, dtype: object

In [248]:
Whis_df.iloc[5215, Whis_df.columns.get_loc('percent_abv')] = 46

Whis_df.iloc[5215]

name                        Wemyss Malts ‘Caribbean Fruits,’ 46%
score                                                         84
price_usd                                                    127
category                                      Single Malt Scotch
review         \nWemyss Malts names the sources of these sing...
author                                               Gavin Smith
sale_volume                                                  750
percent_abv                                                   46
Name: 5217, dtype: object

In [249]:
Whis_df.iloc[5221, Whis_df.columns.get_loc('percent_abv')] = 46

Whis_df.iloc[5221]

name                              Edradour 'Ballechin (#4),' 46%
score                                                         84
price_usd                                                    100
category                                      Single Malt Scotch
review         \nThe fourth release of peated Edradour whisky...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   46
Name: 5223, dtype: object

In [250]:
Whis_df.iloc[5359, Whis_df.columns.get_loc('percent_abv')] = 56.1

Whis_df.iloc[5359]

name           Duncan Taylor (distilled at Glen Grant) 1972 V...
score                                                         83
price_usd                                                    155
category                                      Single Malt Scotch
review         \nI don’t normally like to add water to old wh...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 56.1
Name: 5361, dtype: object

In [251]:
Whis_df.iloc[5422, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[5422]

name                                   The Macallan Whisky Maker
score                                                         83
price_usd                                                     90
category                                      Single Malt Scotch
review         \nFull gold. Oily in texture, with ripe barley...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 5424, dtype: object

In [252]:
Whis_df.iloc[5456, Whis_df.columns.get_loc('percent_abv')] = 43

Whis_df.iloc[5456]

name                          The Speyside, 12 year old, 43% ABV
score                                                         82
price_usd                                                     30
category                                      Single Malt Scotch
review         \nA relative newcomer to the Scotch whisky ind...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   43
Name: 5458, dtype: object

In [253]:
Whis_df.iloc[5464, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[5464]

name                               Tormore, 12 year old, 40% ABV
score                                                         82
price_usd                                                     42
category                                      Single Malt Scotch
review         \nSoft and creamy in texture. A delicate mélan...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 5466, dtype: object

In [254]:
Whis_df.iloc[5467, Whis_df.columns.get_loc('percent_abv')] = 56

Whis_df.iloc[5467]

name           Gordon & MacPhail Single Cask #5054 (distilled...
score                                                         82
price_usd                                                     65
category                                      Single Malt Scotch
review         \nHere’s a whisky not seen very often in the U...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   56
Name: 5469, dtype: object

In [255]:
Whis_df.iloc[5479, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[5479]

name           Gordon & MacPhail (distilled at Glenlossie), 2...
score                                                         82
price_usd                                                     90
category                                      Single Malt Scotch
review         \nFirm on the palate and quite mature. Good ba...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 5481, dtype: object

In [256]:
Whis_df.iloc[5540, Whis_df.columns.get_loc('percent_abv')] = 55.7

Whis_df.iloc[5540]

name           Blackadder Raw Cask #6437 (distilled at Glen G...
score                                                         81
price_usd                                                    175
category                                      Single Malt Scotch
review         \nFrom a sherry hogshead, which is confirmed b...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 55.7
Name: 5542, dtype: object

In [257]:
Whis_df.iloc[5556, Whis_df.columns.get_loc('percent_abv')] = 53.6

Whis_df.iloc[5556]

name           Hart Brothers (distilled at Glenturret), 10 ye...
score                                                         81
price_usd                                                     63
category                                      Single Malt Scotch
review         \nVery fragrant and floral, with bright notes ...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 53.6
Name: 5558, dtype: object

In [258]:
Whis_df.iloc[5560, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[5560]

name           Berry Brothers & Rudd (distilled at Cragganmor...
score                                                         81
price_usd                                                     93
category                                      Single Malt Scotch
review         \nLight straw. Initially this is quite hot and...
author                                                Dave Broom
sale_volume                                                  750
percent_abv                                                   40
Name: 5562, dtype: object

In [259]:
Whis_df.iloc[5578, Whis_df.columns.get_loc('percent_abv')] = 59.2

Whis_df.iloc[5578]

name           Gordon & MacPhail Single Cask #1535 (distilled...
score                                                         80
price_usd                                                     80
category                                      Single Malt Scotch
review         \nA very traditional Rosebank-aged in a refill...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                 59.2
Name: 5580, dtype: object

In [260]:
Whis_df.iloc[5721, Whis_df.columns.get_loc('percent_abv')] = 46.1

Whis_df.iloc[5721]

name                               Mackmyra Swedish whisky 46.1%
score                                                         88
price_usd                                                     55
category                              Single Malt Swedish Whisky
review         \nMackmyra has at long last reached the United...
author                                           Dominic Roskrow
sale_volume                                                  750
percent_abv                                                 46.1
Name: 5723, dtype: object

In [261]:
Whis_df.iloc[5805, Whis_df.columns.get_loc('percent_abv')] = 40

Whis_df.iloc[5805]

name                                                 Blackadder 
score                                                         80
price_usd                                                     90
category                                 Unspecified Single Malt
review         \nVery pale gold color. Aggressive notes of ta...
author                                              John Hansell
sale_volume                                                  750
percent_abv                                                   40
Name: 5807, dtype: object

In [262]:
Whis_df['percent_abv'] = pd.to_numeric(Whis_df['percent_abv'])

Whis_df.dtypes

name            object
score            int64
price_usd      float64
category        object
review          object
author          object
sale_volume     object
percent_abv    float64
dtype: object

In [263]:
Whis_df.head()

,name,score,price_usd,category,review,author,sale_volume,percent_abv
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45.0
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40.0
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50.0
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45.0


In [265]:
Whis_df

,name,score,price_usd,category,review,author,sale_volume,percent_abv
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45.0
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40.0
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50.0
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45.0
...,...,...,...,...,...,...,...,...
5900,World Whiskey Society Goalong 5 year old Bourb...,86,150.0,World Whisky,\n\nThe first Chinese single malt to be export...,Susannah Skiver Barton,750,40.0
5901,"Sierra Norte Single Barrel White, 45%",86,50.0,World Whisky,\nWhite has the most familiar corn whiskey cha...,Jonny McCormick,750,45.0
5902,"Sierra Norte Single Barrel Purple Corn, 45%",85,50.0,World Whisky,\nPepper and allspice light up with savory bri...,Jonny McCormick,750,45.0
5903,That Boutique-y Whisky Company (distilled at S...,85,90.0,World Whisky,\nWhite pepper-dappled summer fruits on the no...,Jonny McCormick,750,52.5


In [266]:
Whis_df['name'].value_counts

<bound method IndexOpsMixin.value_counts of 0       Spirit Hound Single Barrel Straight Malt (Barr...
1                         Dogfish Head Straight Malt, 40%
2          Balcones Texas Bock Straight Malt Whiskey, 50%
3           Woodford Reserve Straight Malt Whiskey, 45.2%
4       Spirit Hound Colorado Honey Malt (Barrel No. 4...
                              ...                        
5900    World Whiskey Society Goalong 5 year old Bourb...
5901                Sierra Norte Single Barrel White, 45%
5902          Sierra Norte Single Barrel Purple Corn, 45%
5903    That Boutique-y Whisky Company (distilled at S...
5904             Abasolo 100% Ancestral Corn Whiskey, 43%
Name: name, Length: 5903, dtype: object>

In [267]:
Whis_df.drop_duplicates(inplace = True)

Whis_df

,name,score,price_usd,category,review,author,sale_volume,percent_abv
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45.0
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40.0
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50.0
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45.0
...,...,...,...,...,...,...,...,...
5900,World Whiskey Society Goalong 5 year old Bourb...,86,150.0,World Whisky,\n\nThe first Chinese single malt to be export...,Susannah Skiver Barton,750,40.0
5901,"Sierra Norte Single Barrel White, 45%",86,50.0,World Whisky,\nWhite has the most familiar corn whiskey cha...,Jonny McCormick,750,45.0
5902,"Sierra Norte Single Barrel Purple Corn, 45%",85,50.0,World Whisky,\nPepper and allspice light up with savory bri...,Jonny McCormick,750,45.0
5903,That Boutique-y Whisky Company (distilled at S...,85,90.0,World Whisky,\nWhite pepper-dappled summer fruits on the no...,Jonny McCormick,750,52.5


In [268]:
Whis_df.to_csv('desktop/independant_projects/whiskey_data_scraper/clean_whiskey_data2.csv', index = False, encoding = 'utf-8-sig')